# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key



In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,haapsalu,58.9431,23.5414,-11.16,82,4,4.11,EE,1707519100
1,1,albany,42.6001,-73.9662,10.23,64,11,0.89,US,1707519101
2,2,beshkent,38.8214,65.6531,7.10,45,0,2.06,UZ,1707519101
3,3,aykhal,66.0000,111.5000,-33.32,100,100,1.71,RU,1707519101
4,4,menongue,-14.6585,17.6910,18.14,94,29,1.06,AO,1707519101


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
cities_map_1 = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM")

# Display the map
cities_map_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_df = city_data_df.loc[(city_data_df["Humidity"]<=40)&(city_data_df["Max Temp"]<=32) & (city_data_df["Max Temp"]>=-10)]


# Drop any rows with null values
perfect_new_df = perfect_df.dropna(how="any")

perfect_new_df.count()

# Display sample data
perfect_new_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,ciudad sabinas hidalgo,26.5000,-100.1667,26.73,40,68,3.82,MX,1707519105
37,37,akcaabat,41.0212,39.5715,17.93,29,0,6.17,TR,1707519106
61,61,mirabad,30.4362,61.8383,13.01,21,0,1.47,AF,1707519110
91,91,as samawah,31.3320,45.2944,16.98,32,100,4.83,IQ,1707519114
127,127,san,13.3034,-4.8956,27.23,8,100,3.69,ML,1707518949


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_new_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(1,"Hotel Name", "")


# Display sample data
hotel_df.head()

,City,Hotel Name,Country,Lat,Lng,Humidity
30,ciudad sabinas hidalgo,,MX,26.5000,-100.1667,40
37,akcaabat,,TR,41.0212,39.5715,29
61,mirabad,,AF,30.4362,61.8383,21
91,as samawah,,IQ,31.3320,45.2944,32
127,san,,ML,13.3034,-4.8956,8


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"
filters = ""
bias = ""
params = { 
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row[4]
    latitude = row[3]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ciudad sabinas hidalgo - nearest hotel: La Fogata
akcaabat - nearest hotel: Nazar Otel
mirabad - nearest hotel: No hotel found
as samawah - nearest hotel: فندق قصر الغدير
san - nearest hotel: La Roche
odienne - nearest hotel: Hotel Frontière
tamanrasset - nearest hotel: فندق الأمان
crane - nearest hotel: No hotel found
loreto - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
gao - nearest hotel: No hotel found
bossangoa - nearest hotel: No hotel found
faya-largeau - nearest hotel: No hotel found
buka - nearest hotel: No hotel found
niamey - nearest hotel: Redidence Croix du Sud
tank - nearest hotel: No hotel found
ubari - nearest hotel: فندق اوباري
coyhaique - nearest hotel: No hotel found
port augusta - nearest hotel: Hotel Commonwealth
i-n-salah - nearest hotel: باجودة
illizi - nearest hotel: بوناقة
uyuni - nearest hotel: Hostel Jerian
betare oya - nearest hotel: No hotel found
rouissat - nearest hotel: فندق الأروي الذهبي
anar darah - neares

,City,Hotel Name,Country,Lat,Lng,Humidity
30,ciudad sabinas hidalgo,La Fogata,MX,26.5000,-100.1667,40
37,akcaabat,Nazar Otel,TR,41.0212,39.5715,29
61,mirabad,No hotel found,AF,30.4362,61.8383,21
91,as samawah,فندق قصر الغدير,IQ,31.3320,45.2944,32
127,san,La Roche,ML,13.3034,-4.8956,8
134,odienne,Hotel Frontière,CI,9.5100,-7.5692,15
147,tamanrasset,فندق الأمان,DZ,22.7850,5.5228,16
164,crane,No hotel found,US,31.3974,-102.3501,18
184,loreto,No hotel found,MX,22.2667,-101.9667,11
186,at taj,No hotel found,LY,24.2049,23.2857,32


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols=["City", "Country", "Hotel Name"])

# Display the map
hotels_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)